In [233]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random

In [238]:
class ShowerEnv(gym.Env):

    def __init__(self):

        #Actions: down, stay, up
        self.action_space = Discrete(3)
        #Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]), dtype=float)

        #Set start temp
        #self.state = 38 + random.randint(-3, 3)

        # Set shower length
        self.shower_length = 60

    def step(self, action):
        self.state += action - 1
        self.shower_length -= 1

        if 37 <= self.state <= 39:
            reward = 1
        else:
            if self.state < 37:
                reward = -(37 - self.state)
            else:
                reward = (self.state - 39)

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        self.state +=  -1 + random.random()*(1 - (-1))

        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        #Set start temp
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state
        pass


In [239]:
env = ShowerEnv()

C:\Users\andre\PycharmProjects\RL_Module\venv\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [240]:
env.observation_space.sample()

array([46.71726774])

In [241]:
episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score {}'.format(episode, score))

Episode: 1 Score 497.1953818248798
Episode: 2 Score -189.74274746121955
Episode: 3 Score 72.09828653492463
Episode: 4 Score -426.0845281580085
Episode: 5 Score -129.21173376626223
Episode: 6 Score 28.610603328575387
Episode: 7 Score 232.98224296217884
Episode: 8 Score -364.8290969336085
Episode: 9 Score -0.22339441928889414
Episode: 10 Score -110.83700983488538


In [242]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [243]:
states = env.observation_space.shape
actions = env.action_space.n

In [244]:
actions

3

In [245]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape = states))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [246]:
model = build_model(states, actions)

In [247]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 128)               256       
                                                                 
 dense_40 (Dense)            (None, 64)                8256      
                                                                 
 dense_41 (Dense)            (None, 3)                 195       
                                                                 
Total params: 8,707
Trainable params: 8,707
Non-trainable params: 0
_________________________________________________________________


In [248]:
from tensorflow.keras.optimizers.legacy import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [249]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn =DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [250]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 31:31 - reward: -3.0000

C:\Users\andre\PycharmProjects\RL_Module\venv\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\andre\PycharmProjects\RL_Module\venv\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 38s 4ms/step - reward: 10.3814
166 episodes - episode_reward: 625.409 [-656.097, 1968.326] - loss: 5794.424 - mae: 402.775 - mean_q: 616.879

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 41s 4ms/step - reward: 0.4341
167 episodes - episode_reward: 25.573 [-637.624, 787.161] - loss: 10009.078 - mae: 627.729 - mean_q: 949.742

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 0.4249
167 episodes - episode_reward: 25.843 [-543.868, 779.743] - loss: 8516.475 - mae: 585.737 - mean_q: 884.716

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: 0.0938
166 episodes - episode_reward: 5.967 [-526.399, 548.448] - loss: 8231.350 - mae: 606.166 - mean_q: 916.857

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: -0.1154
done, took 209.879 seconds


In [251]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: ShowerEnv.render() got an unexpected keyword argument 'mode'

In [252]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 2018.483, steps: 60
Episode 2: reward: 1882.412, steps: 60
Episode 3: reward: 1836.541, steps: 60
Episode 4: reward: 1635.352, steps: 60
Episode 5: reward: 1882.205, steps: 60
Episode 6: reward: 2145.592, steps: 60
Episode 7: reward: 1664.948, steps: 60
Episode 8: reward: 1893.519, steps: 60
Episode 9: reward: 1537.306, steps: 60
Episode 10: reward: 1681.470, steps: 60
Episode 11: reward: 1490.194, steps: 60
Episode 12: reward: 1896.565, steps: 60
Episode 13: reward: 1485.979, steps: 60
Episode 14: reward: 1691.272, steps: 60
Episode 15: reward: 1768.341, steps: 60
Episode 16: reward: 1572.377, steps: 60
Episode 17: reward: 1756.956, steps: 60
Episode 18: reward: 1831.596, steps: 60
Episode 19: reward: 1531.223, steps: 60
Episode 20: reward: 1446.817, steps: 60
Episode 21: reward: 1892.028, steps: 60
Episode 22: reward: 1967.482, steps: 60
Episode 23: reward: 1916.352, steps: 60
Episode 24: reward: 1747.703, steps: 60
Episode 25: reward: 